In [1]:
# import pandas as pd
# import numpy as np
# from sentence_transformers import SentenceTransformer
# from tqdm import tqdm  # For progress bars
# import psycopg2


# # Initialize the model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Load the DataFrame
# jd_df = pd.read_csv('../data/jd_data/jd.csv')

# # Replace NaN and non-string values with empty strings
# for column in ['work_experience', 'education', 'skills']:
#     jd_df[column] = jd_df[column].fillna('').astype(str)

# # Generate embeddings for each column with a progress bar
# tqdm.pandas()

# # Generate embeddings and store them as lists
# jd_df['work_experience_embedding'] = model.encode(jd_df['work_experience'].tolist(), show_progress_bar=True).tolist()
# jd_df['education_embedding'] = model.encode(jd_df['education'].tolist(), show_progress_bar=True).tolist()
# jd_df['skills_embedding'] = model.encode(jd_df['skills'].tolist(), show_progress_bar=True).tolist()

# jd_df.head()


c:\Users\deepa\Downloads\cv matching\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 27/27 [00:18<00:00,  1.43it/s]


,company_name,job_description,position_title,description_length,model_response,work_experience,education,skills,work_experience_word_count,education_word_count,skills_word_count,work_experience_embedding,education_embedding,skills_embedding
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo...",Bachelors degree or equivalent practical exper...,"Ability to plan, pitch and execute a territory...",years of experience in SaaS or productivity to...,6,129,53,"[0.04428974911570549, -0.020102469250559807, 0...","[0.011864776723086834, -0.05470835790038109, 0...","[0.018535442650318146, -0.020643938332796097, ..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ...",,A passion to help people understand how Apple ...,As an ASC you will be highly influential in gr...,0,63,63,"[-0.11883842200040817, 0.04829854518175125, -0...","[0.00019024196080863476, 0.044248033314943314,...","[-0.02461927942931652, -0.07895708829164505, -..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus...",Understanding of category manufacturing and sa...,Excellent communicator. Alignment with our com...,demonstrated years of experience in preferabl...,15,89,46,"[0.013472531922161579, -0.0417313314974308, -0...","[0.041221898049116135, -0.02589244581758976, -...","[0.017600223422050476, -0.02048618160188198, 0..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs...",,"Experience in creating wireframes, designing m...",web designers looking to expand your professio...,0,58,190,"[-0.11883842200040817, 0.04829854518175125, -0...","[0.011584104970097542, -0.008616739884018898, ...","[-0.046512532979249954, -0.016284015029668808,..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo...",Not specified in the job description.,"Years of experience with HTML and CSS/SASS, ye...",At trackfive we've got big goals. We're on a m...,6,80,226,"[-0.08566185086965561, 0.0067263273522257805, ...","[-0.025381682440638542, -0.00852982047945261, ...","[-0.04066207632422447, -0.07478011399507523, -..."


In [4]:
# # Connect to PostgreSQL
# conn = psycopg2.connect(
#     dbname="vector_db",
#     user="postgres",
#     password="pass123",
#     host="localhost",
#     port="5432"
# )
# cursor = conn.cursor()

# # Insert data into the database
# for _, row in jd_df.iterrows():
#     cursor.execute("""
#         INSERT INTO job_embeddings (
#             company_name, job_description, work_experience_embedding, education_embedding, skills_embedding
#         ) VALUES (%s, %s, %s, %s, %s)
#     """, (
#         row['company_name'],
#         row['job_description'],
#         row['work_experience_embedding'],
#         row['education_embedding'],
#         row['skills_embedding']
#     ))

# # Commit and close connection
# conn.commit()
# cursor.close()
# conn.close()

# print("Embeddings stored successfully!")


Embeddings stored successfully!


In [1]:
import pandas as pd
import psycopg2
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_query_embeddings(work_experience, education, skills):
    return {
        "work_experience_embedding": model.encode(work_experience),
        "education_embedding": model.encode(education),
        "skills_embedding": model.encode(skills)
    }

# Function to insert job data into the database
def insert_job_data_from_df(jd_df, db_config):
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Fill NaN values with empty strings
    jd_df = jd_df.fillna('')

    for _, job in jd_df.iterrows():
        embeddings = generate_query_embeddings(
            job['work_experience'],
            job['education'],
            job['skills']
        )
        cursor.execute("""
            INSERT INTO job_embeddings (
                company_name, job_description, work_experience_embedding, education_embedding, skills_embedding
            ) VALUES (%s, %s, %s::vector, %s::vector, %s::vector)
        """, (
            job['company_name'],
            job['job_description'],
            embeddings['work_experience_embedding'].tolist(),  # Convert to list
            embeddings['education_embedding'].tolist(),        # Convert to list
            embeddings['skills_embedding'].tolist()            # Convert to list
        ))

    conn.commit()
    cursor.close()
    conn.close()
    print("Job data from DataFrame inserted successfully!")

# Load job data from CSV file
jd_df = pd.read_csv('../data/jd_data/jd.csv')

# Database configuration
db_config = {
    "dbname": "vector_db",
    "user": "postgres",
    "password": "pass123",
    "host": "localhost",
    "port": "5432"
}

# Insert job data from DataFrame
insert_job_data_from_df(jd_df, db_config)


c:\Users\deepa\Downloads\cv matching\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Job data from DataFrame inserted successfully!


In [3]:
def query_top_k_similar(work_experience, education, skills, db_config, weights=None, k=5):
    if weights is None:
        weights = {"work_experience": 1.0, "education": 1.0, "skills": 1.0}

    embeddings = generate_query_embeddings(work_experience, education, skills)

    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Prepare embeddings as lists for PostgreSQL compatibility
    work_experience_embedding = embeddings['work_experience_embedding'].tolist()
    education_embedding = embeddings['education_embedding'].tolist()
    skills_embedding = embeddings['skills_embedding'].tolist()

    query = """
    SELECT 
        id,
        company_name,
        job_description,
        (
            (%s * (work_experience_embedding <-> %s::vector)) +
            (%s * (education_embedding <-> %s::vector)) +
            (%s * (skills_embedding <-> %s::vector))
        ) AS combined_distance
    FROM 
        job_embeddings
    ORDER BY 
        combined_distance ASC  -- Order by ascending distance
    LIMIT %s;
    """

    cursor.execute(query, (
        weights['work_experience'], work_experience_embedding,
        weights['education'], education_embedding,
        weights['skills'], skills_embedding,
        k
    ))

    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return results

# Example usage
top_k = query_top_k_similar(
    work_experience="5 years in data analysis",
    education="Masters in Data Science",
    skills="Python, R, SQL",
    db_config=db_config,
    weights={"work_experience": 1.0, "education": 1.0, "skills": 1.0},
    k=5
)

# Display the results
for result in top_k:
    print(f"ID: {result[0]}, Company: {result[1]}, Distance: {result[3]}")


ID: 466, Company: StrongArm Technologies, Distance: 3.237159275358098
ID: 773, Company: Brains Workgroup, Distance: 3.503761709773543
ID: 464, Company: Montblanc, Distance: 3.5740234359768372
ID: 418, Company: Tempus, Distance: 3.5842316046693403
ID: 467, Company: Facebook App, Distance: 3.595168533381713


In [45]:
# import os
# import json
# import pdfplumber
# from dotenv import load_dotenv
# from tqdm import tqdm  # Import tqdm for progress tracking
# import openai  # Ensure this library is installed and properly configured
# import pandas as pd
# # Load environment variables
# load_dotenv()

# def extract_information(pdf_path):
#     with pdfplumber.open(pdf_path) as pdf:
#         resume_text = ""
#         for page in pdf.pages:
#             resume_text = " ".join([resume_text, page.extract_text()])
#     resume_text = resume_text.strip()
#     return resume_text

# def process_with_chatgpt(text):
#     openai.api_key = os.getenv("OPENAI_API_KEY")
    
#     schema_example = json.dumps({
#         "basics": {
#             "name": "John Doe",
#             "label": "Programmer",
#             "image": "",
#             "email": "john@gmail.com",
#             "phone": "(912) 555-4321",
#             "url": "https://johndoe.com",
#             "summary": "A summary of John Doe…",
#             "location": {
#                 "address": "2712 Broadway St",
#                 "postalCode": "CA 94115",
#                 "city": "San Francisco",
#                 "countryCode": "US",
#                 "region": "California"
#             },
#             "profiles": [ {
#                 "network": "Twitter",
#                 "username": "john",
#                 "url": "https://twitter.com/john"
#             }]
#         },
#         "work": [ {
#             "name": "Company",
#             "position": "President",
#             "url": "https://company.com",
#             "startDate": "2013-01-01",
#             "endDate": "2014-01-01",
#             "summary": "Description…",
#             "highlights": [
#                 "Started the company"
#             ]
#         }],
#         "volunteer": [ {
#             "organization": "Organization",
#             "position": "Volunteer",
#             "url": "https://organization.com/",
#             "startDate": "2012-01-01",
#             "endDate": "2013-01-01",
#             "summary": "Description…",
#             "highlights": [
#                 "Awarded 'Volunteer of the Month'"
#             ]
#         }],
#         "education": [ {
#             "institution": "University",
#             "url": "https://institution.com/",
#             "area": "Software Development",
#             "studyType": "Bachelor",
#             "startDate": "2011-01-01",
#             "endDate": "2013-01-01",
#             "score": "4.0",
#             "courses": [
#                 "DB1101 - Basic SQL"
#             ]
#         }],
#         "awards": [ {
#             "title": "Award",
#             "date": "2014-11-01",
#             "awarder": "Company",
#             "summary": "There is no spoon."
#         }],
#         "certificates": [ {
#             "name": "Certificate",
#             "date": "2021-11-07",
#             "issuer": "Company",
#             "url": "https://certificate.com"
#         }],
#         "publications": [ {
#             "name": "Publication",
#             "publisher": "Company",
#             "releaseDate": "2014-10-01",
#             "url": "https://publication.com",
#             "summary": "Description…"
#         }],
#         "skills": [ {
#             "name": "Web Development",
#             "level": "Master",
#             "keywords": [
#                 "HTML",
#                 "CSS",
#                 "JavaScript"
#             ]
#         }],
#         "languages": [ {
#             "language": "English",
#             "fluency": "Native speaker"
#         }],
#         "interests": [ {
#             "name": "Wildlife",
#             "keywords": [
#                 "Ferrets",
#                 "Unicorns"
#             ]
#         }],
#         "references": [ {
#             "name": "Jane Doe",
#             "reference": "Reference…"
#         }],
#         "projects": [ {
#             "name": "Project",
#             "startDate": "2019-01-01",
#             "endDate": "2021-01-01",
#             "description": "Description...",
#             "highlights": [
#                 "Won award at AIHacks 2016"
#             ],
#             "url": "https://project.com/"
#         }]
#     }, indent=2)
    
#     prompt = f"""
#     Convert the given resume text into a structured JSON format strictly adhering to the provided schema.  
#     - If a field is not explicitly mentioned in the resume, leave it empty.   
#     - Return only the JSON object without additional text or commentary.  

#     Schema:  
#     {schema_example}  

#     Resume Text:  
#     {text}  
# """

#     client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": prompt}
#         ]
#     )
#     result = response.choices[0].message.content
#     return result

# def extract_top_k(pdf_path):
#     text = extract_information(pdf_path)
#     structured_data = process_with_chatgpt(text)
#     cv_data = json.loads(structured_data)
#     work_paragraph = ""
#     for work in cv_data.get('work', []):
#         work_summary = work.get('summary', '')
#         work_highlights = ', '.join(work.get('highlights', []))
#         if work_summary or work_highlights:
#             work_paragraph += f"Position: {work.get('position', '')}\n"
#             work_paragraph += f"Company: {work.get('name', '')}\n"
#             work_paragraph += f"Summary: {work_summary}\n"
#             if work_highlights:
#                 work_paragraph += f"Highlights: {work_highlights}\n"
#             work_paragraph += "\n"

#     # Extract 'education' section and combine them into a paragraph
#     education_paragraph = ""
#     for education in cv_data.get('education', []):
#         institution = education.get('institution', '')
#         area = education.get('area', '')
#         study_type = education.get('studyType', '')
#         start_date = education.get('startDate', '')
#         end_date = education.get('endDate', '')
#         courses = ', '.join(education.get('courses', []))

#         education_paragraph += f"Institution: {institution}\n"
#         education_paragraph += f"Area of Study: {area}\n"
#         education_paragraph += f"Degree: {study_type}\n"
#         education_paragraph += f"Start Date: {start_date}\n"
#         education_paragraph += f"End Date: {end_date}\n"
#         if courses:
#             education_paragraph += f"Courses: {courses}\n"
#         education_paragraph += "\n"

#     # Extract 'skills' section and combine them into a paragraph
#     skills_projects_paragraph = ""
#     for skills in cv_data.get('skills', []):
#         skill_name = skills.get('name', '')
#         skill_level = skills.get('level', '')
#         skill_keywords = ', '.join(skills.get('keywords', []))

#         skills_projects_paragraph += f"Skill: {skill_name}\n"
#         if skill_level:
#             skills_projects_paragraph += f"Level: {skill_level}\n"
#         if skill_keywords:
#             skills_projects_paragraph += f"Keywords: {skill_keywords}\n"
#         skills_projects_paragraph += "\n"

#     # Create a DataFrame with 'id', 'folder_name', 'work', 'education', and 'skills_projects' columns
#     df = pd.DataFrame({
#         'id': pdf_path,
#         'work': [work_paragraph],
#         'education': [education_paragraph],
#         'skills_projects': [skills_projects_paragraph]
#     })
#     df.fillna('',inplace=True)
#     results=query_top_k_similar_job_description(df['work'], df['education'], df['skills_projects'], db_config)



#     return results



In [46]:
# db_config = {
#     "dbname": "vector_db",
#     "user": "postgres",
#     "password": "pass123",
#     "host": "localhost",
#     "port": "5432"
# }


# # Function to generate embeddings
# def generate_query_embeddings(work_experience, education, skills):
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     return {
#         "work_experience_embedding": model.encode(work_experience),
#         "education_embedding": model.encode(education),
#         "skills_embedding": model.encode(skills)
#     }
# def query_top_k_similar_job_description(work_experience, education, skills, db_config, weights=None, k=5):
#     if weights is None:
#         weights = {"work_experience": 1.0, "education": 1.0, "skills": 1.0}

#     embeddings = generate_query_embeddings(work_experience, education, skills)

#     conn = psycopg2.connect(**db_config)
#     cursor = conn.cursor()

#     # Prepare embeddings as lists for PostgreSQL compatibility
#     work_experience_embedding = embeddings['work_experience_embedding'].tolist()[0]
#     education_embedding = embeddings['education_embedding'].tolist()[0]
#     skills_embedding = embeddings['skills_embedding'].tolist()[0]

#     print("Work experience embedding:", work_experience_embedding)
#     print("Education embedding:", education_embedding)
#     print("Skills embedding:", skills_embedding)

#     query = """
#     SELECT 
#         id,
#         company_name,
#         job_description,
#         (
#             (%s * (work_experience_embedding <-> %s::vector)) +
#             (%s * (education_embedding <-> %s::vector)) +
#             (%s * (skills_embedding <-> %s::vector))
#         ) AS combined_distance
#     FROM 
#         job_embeddings
#     ORDER BY 
#         combined_distance ASC  -- Order by ascending distance
#     LIMIT %s;
#     """

#     cursor.execute(query, (
#         weights['work_experience'], work_experience_embedding,
#         weights['education'], education_embedding,
#         weights['skills'], skills_embedding,
#         k
#     ))

#     results = cursor.fetchall()
#     cursor.close()
#     conn.close()
#     return results
